Fetch Posts from Firestore

In [1]:
import openai  # Or any embedding library like transformers
import numpy as np
from scipy.spatial.distance import cosine
import os
from dotenv import load_dotenv

load_dotenv()
# Define your OpenAI API key if using OpenAI embeddings
openai.api_key =os.getenv("OPENAI_KEY")




In [2]:
#Initiate firebase client
import firebase_admin
from firebase_admin import credentials, firestore

# Path to your service account key
cred = credentials.Certificate('creds/sensenets-dataset-firebase-adminsdk-rpero-9c552cac56.json')
firebase_admin.initialize_app(cred)

# Initialize Firestore DB
db = firestore.client()


In [69]:

def fetch_posts(collection_name):
    """Fetch posts from Firestore collection."""
    posts = []
    docs = db.collection(collection_name).limit(200).stream()
    for doc in docs:
        posts.append((doc.to_dict(),doc.id))
    return posts

def process_posts(posts:list):
    processed_posts = []
    for post in posts:
        content, url = get_thread(post[0]['generic']['thread'])
        id = post[1]
        processed_posts.append(
            {
                "content":content,
                "id":id
            }
        )
    return processed_posts 
def get_thread(thread_list:list):
    full_text = ''
    url = thread_list[0]["url"]
    for post_dict in thread_list:
        full_text = f'{full_text} {post_dict["content"]}'
    return full_text,url

def get_embedding(text, model="text-embedding-3-small"):
    """Generate embedding for given text using the updated OpenAI API."""
    if not text or not isinstance(text, str):
        raise ValueError("Input text must be a non-empty string.")
    response = openai.embeddings.create(input=[text], model=model).data[0].embedding[:256]
    return response


def embed_posts(posts):
    """Generate embeddings for each post."""
    embeddings = {}
    for post in posts:
        post_id = post['id'] # Adjust based on your document structure
        content = post['content']
        embeddings[post_id] = get_embedding(content)
    return embeddings

def embed_keywords(keywords):
    """Generate embeddings for each keyword with added context if necessary."""
    keyword_embeddings = {}
    for keyword in keywords:
        keyword_with_context = f"{keyword['name'], keyword['context']} - relevant context here"  # Add any context if needed
        keyword_embeddings[keyword['name']] = get_embedding(keyword_with_context)
    return keyword_embeddings

In [70]:
get_embedding("I love the stars as they remind me how insignificant is humane stupidity")

[0.02592855878174305,
 -0.03406156599521637,
 -0.029164113104343414,
 0.014295268803834915,
 0.012346582487225533,
 -0.056886933743953705,
 -0.015354177914559841,
 0.013376077637076378,
 0.01146415900439024,
 0.016310136765241623,
 -0.020178096368908882,
 -0.0018264338141307235,
 -0.0737118199467659,
 -0.013596683740615845,
 0.06729953736066818,
 -1.0484529866516823e-06,
 -0.050415825098752975,
 -0.03091425448656082,
 0.06024014577269554,
 0.06147553771734238,
 0.008537452667951584,
 0.02739926613867283,
 0.023016558960080147,
 0.05023933947086334,
 -0.0018540095770731568,
 0.003595877904444933,
 0.011022946797311306,
 0.02011926658451557,
 0.09700781106948853,
 0.007110866717994213,
 0.009353694505989552,
 -0.024899063631892204,
 0.005511473398655653,
 -0.019633933901786804,
 -0.030178900808095932,
 0.032826174050569534,
 -0.011427391320466995,
 -0.06594648957252502,
 -0.015471834689378738,
 0.0034230698365718126,
 0.007831512950360775,
 -0.0029101609252393246,
 0.04306229203939438,
 

In [71]:
posts=fetch_posts('posts')
posts=process_posts(posts)[0:40]


In [72]:
posts_dict = {post['id']:post['content'] for post in posts}

In [14]:
def find_similar_posts(post_embeddings, keyword_embeddings, threshold=0.8):
    """Find similar posts for each keyword based on cosine similarity."""
    keyword_matches = {keyword: [] for keyword in keyword_embeddings}

    for post_id, post_emb in post_embeddings.items():
        for keyword, keyword_emb in keyword_embeddings.items():
            similarity = 1 - cosine(post_emb, keyword_emb)  # Cosine similarity
            if similarity > threshold:
                keyword_matches[keyword].append((post_id, similarity))

    return keyword_matches

In [73]:

post_embeddings = embed_posts(posts)




In [74]:
keywords = [
    {
        "name":"AI",
        "context":"Artificial Intelligence on all its aspects, algo, theory, philosophy, literiture "
    },
    {
        'name':"Crypto",
        'context':'cryptocurrencies, bitcoin, ehtereum'
    },
    {
      'name':'OpenScience',
      'context':'the field of open source science where data is open, sharing datasets, open access is avilable. Desentralized science, science for good  '  
    },
    {
        'name':"politics",
        'context':''

    },
    {
        'name':'climate change',
        'context':'global warming, natural disasters, co2 '
    },
    {
        'name':'social media',
        'context':'social digital platforms like facebook, twitter, and etc..'
    } 
    ] 
keyword_embeddings = embed_keywords(keywords)


In [75]:
# Match each post to keywords one at a time
for post_id, post_emb in post_embeddings.items():
    
    for keyword, keyword_emb in keyword_embeddings.items():
        similarity = 1 - cosine(post_emb, keyword_emb)
        #print(f"Keyword: {keyword}, Similarity: {similarity}")
        if similarity > 0.8:  # Adjust threshold based on results
            print(f"\nPost ID: {post_id}")
            print(posts_dict[post_id]) 
            print(f"Matched Keyword: {keyword}, Similarity: {similarity}")
            print('++++++++++++++++')

In [76]:
# Match each post to keywords one at a time
for post_id, post_emb in post_embeddings.items():
    print(f"\nPost ID: {post_id}")
    print(posts_dict[post_id])
    for keyword, keyword_emb in keyword_embeddings.items():
        similarity = 1 - cosine(post_emb, keyword_emb)
        print(f"Keyword: {keyword}, Similarity: {similarity}")
        if similarity > 0.8:  # Adjust threshold based on results
            
            print(f"Matched Keyword: {keyword}, Similarity: {similarity}")
            print('++++++++++++++++')


Post ID: 00iucJrZjHiHhUQVLlag
 this may very well be a generational bookhttps://www.plurality.net
Keyword: AI, Similarity: 0.353927184842329
Keyword: Crypto, Similarity: 0.2409011311382907
Keyword: OpenScience, Similarity: 0.30435508663583444
Keyword: politics, Similarity: 0.2985378220715609
Keyword: climate change, Similarity: 0.18590275477920026
Keyword: social media, Similarity: 0.3195280603001227

Post ID: 01B6LCPTI6rsZmcs9wny
 There are still a few places available on our Scientometrics With Open Data course with @cwts and @MsPhelps running week after next in a Europe/Africa friendly timezone. 

We'll be closing the registrations soon so if this is of interest then head to:

https://www.cwts.nl/education/cwts-course-program/scientometrics-using-open-data
Keyword: AI, Similarity: 0.3293751618849292
Keyword: Crypto, Similarity: 0.39150582992389915
Keyword: OpenScience, Similarity: 0.5089523847322832
Keyword: politics, Similarity: 0.3470087905808583
Keyword: climate change, Similari